In [56]:
# For data
import networkx as nx
from parse import *
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [57]:
from IPython.core.display import display, HTML
from IPython.display import Javascript
from string import Template
import json

In [58]:
import os
import pandas as pd

In [59]:
files = []
f = os.listdir('gml_topo/')
for file in f:
    if '.gml' in file:
        files.append(file)
files

['Bandcon.gml',
 'Garr201105.gml',
 'VtlWavenet2011.gml',
 'Belnet2010.gml',
 'Shentel.gml',
 'Easynet.gml',
 'Arpanet19719.gml',
 'Padi.gml',
 'Cudi.gml',
 'KentmanApr2007.gml',
 'Cernet.gml',
 'Cogentco.gml',
 'Garr199905.gml',
 'LambdaNet.gml',
 'GtsRomania.gml',
 'Airtel.gml',
 'Navigata.gml',
 'Geant2010.gml',
 'Renam.gml',
 'Marwan.gml',
 'Chinanet.gml',
 'Zamren.gml',
 'Twaren.gml',
 'Integra.gml',
 'Cesnet1999.gml',
 'CrlNetworkServices.gml',
 'Garr200109.gml',
 'Cesnet200304.gml',
 'Nordu2010.gml',
 'Esnet.gml',
 'Compuserve.gml',
 'Rhnet.gml',
 'Garr201008.gml',
 'Getnet.gml',
 'Garr200212.gml',
 'DeutscheTelekom.gml',
 'Ernet.gml',
 'Kreonet.gml',
 'Garr201005.gml',
 'HiberniaUs.gml',
 'KentmanFeb2008.gml',
 'Istar.gml',
 'Azrena.gml',
 'Cesnet201006.gml',
 'Ulaknet.gml',
 'KentmanJan2011.gml',
 'Arn.gml',
 'Ai3.gml',
 'BtLatinAmerica.gml',
 'Vinaren.gml',
 'Switch.gml',
 'Pern.gml',
 'Abilene.gml',
 'GtsHungary.gml',
 'Renater2010.gml',
 'Harnet.gml',
 'Garr201112.gml',
 'K

In [60]:
data = []
for file in files:
    print(file)
    H = nx.read_graphml('gml_topo/' + file.replace('.gml', '.graphml'))
    data.append([file, len(H.nodes()), len(H.edges())])
#     print(data)

Bandcon.gml
Garr201105.gml
VtlWavenet2011.gml
Belnet2010.gml
Shentel.gml
Easynet.gml
Arpanet19719.gml
Padi.gml
Cudi.gml
KentmanApr2007.gml
Cernet.gml
Cogentco.gml
Garr199905.gml
LambdaNet.gml
GtsRomania.gml
Airtel.gml
Navigata.gml
Geant2010.gml
Renam.gml
Marwan.gml
Chinanet.gml
Zamren.gml
Twaren.gml
Integra.gml
Cesnet1999.gml
CrlNetworkServices.gml
Garr200109.gml
Cesnet200304.gml
Nordu2010.gml
Esnet.gml
Compuserve.gml
Rhnet.gml
Garr201008.gml
Getnet.gml
Garr200212.gml
DeutscheTelekom.gml
Ernet.gml
Kreonet.gml
Garr201005.gml
HiberniaUs.gml
KentmanFeb2008.gml
Istar.gml
Azrena.gml
Cesnet201006.gml
Ulaknet.gml
KentmanJan2011.gml
Arn.gml
Ai3.gml
BtLatinAmerica.gml
Vinaren.gml
Switch.gml
Pern.gml
Abilene.gml
GtsHungary.gml
Renater2010.gml
Harnet.gml
Garr201112.gml
Kdl.gml
Garr201109.gml
Garr201111.gml
Nordu1997.gml
PionierL3.gml
Grnet.gml
Ntelos.gml
Claranet.gml
Packetexchange.gml
Reuna.gml
Renater2008.gml
GtsSlovakia.gml
Janetlense.gml
Globenet.gml
Garr201103.gml
Xspedius.gml
Ibm.gml
BtEuro

In [61]:
data

[['Bandcon.gml', 22, 28],
 ['Garr201105.gml', 59, 84],
 ['VtlWavenet2011.gml', 92, 96],
 ['Belnet2010.gml', 22, 32],
 ['Shentel.gml', 28, 35],
 ['Easynet.gml', 19, 29],
 ['Arpanet19719.gml', 18, 22],
 ['Padi.gml', 15, 6],
 ['Cudi.gml', 51, 52],
 ['KentmanApr2007.gml', 23, 24],
 ['Cernet.gml', 41, 59],
 ['Cogentco.gml', 197, 245],
 ['Garr199905.gml', 23, 25],
 ['LambdaNet.gml', 42, 46],
 ['GtsRomania.gml', 21, 24],
 ['Airtel.gml', 16, 37],
 ['Navigata.gml', 13, 17],
 ['Geant2010.gml', 37, 58],
 ['Renam.gml', 5, 4],
 ['Marwan.gml', 16, 18],
 ['Chinanet.gml', 42, 66],
 ['Zamren.gml', 36, 34],
 ['Twaren.gml', 20, 20],
 ['Integra.gml', 27, 36],
 ['Cesnet1999.gml', 13, 12],
 ['CrlNetworkServices.gml', 33, 38],
 ['Garr200109.gml', 22, 24],
 ['Cesnet200304.gml', 29, 33],
 ['Nordu2010.gml', 18, 17],
 ['Esnet.gml', 68, 92],
 ['Compuserve.gml', 14, 17],
 ['Rhnet.gml', 16, 18],
 ['Garr201008.gml', 55, 74],
 ['Getnet.gml', 7, 8],
 ['Garr200212.gml', 27, 29],
 ['DeutscheTelekom.gml', 39, 62],
 ['Ern

In [62]:
df = pd.DataFrame(data, columns = ['Name', 'Nodes number', 'Edges number'])
df.sort_values(by=['Nodes number'])

,Name,Nodes number,Edges number
187,Arpanet196912.gml,4,4
18,Renam.gml,5,4
212,Dataxchange.gml,6,11
162,Layer42.gml,6,7
103,Epoch.gml,6,7
...,...,...,...
200,Colt.gml,153,191
201,UsCarrier.gml,158,189
129,DialtelecomCz.gml,193,151
11,Cogentco.gml,197,245


In [63]:
H1=nx.read_graphml('gml_topo/UsCarrier.graphml')
H = nx.Graph(H1)
H

In [64]:
len(H.nodes())

158

In [65]:
len(H.edges())

189

In [66]:
for e in H.edges():
    data = H.get_edge_data(e[1], e[0])
    speeds = ['100Mb/s', '200Mb/s', '300Mb/s']
    print(H[e[1]][e[0]])
    H[e[1]][e[0]]["LinkLabel"] = random.choice(speeds)
#     print(data)

{'id': 'e54', 'key': 0}
{'id': 'e56', 'key': 0}
{'id': 'e55', 'key': 0}
{'id': 'e42', 'key': 0}
{'id': 'e41', 'key': 0}
{'id': 'e43', 'key': 0}
{'id': 'e44', 'key': 0}
{'id': 'e27', 'key': 0}
{'id': 'e45', 'key': 0}
{'id': 'e13', 'key': 0}
{'id': 'e48', 'key': 0}
{'id': 'e49', 'key': 0}
{'id': 'e50', 'key': 0}
{'id': 'e46', 'key': 0}
{'id': 'e47', 'key': 0}
{'id': 'e51', 'key': 0}
{'id': 'e52', 'key': 0}
{'id': 'e53', 'key': 0}
{'id': 'e172', 'key': 0}
{'id': 'e173', 'key': 0}
{'id': 'e171', 'key': 0}
{'id': 'e175', 'key': 0}
{'id': 'e174', 'key': 0}
{'id': 'e170', 'key': 0}
{'id': 'e168', 'key': 0}
{'id': 'e177', 'key': 0}
{'id': 'e178', 'key': 0}
{'id': 'e179', 'key': 0}
{'id': 'e176', 'key': 0}
{'id': 'e180', 'key': 0}
{'id': 'e181', 'key': 0}
{'id': 'e63', 'key': 0}
{'id': 'e113', 'key': 0}
{'id': 'e97', 'key': 0}
{'id': 'e117', 'key': 0}
{'id': 'e116', 'key': 0}
{'id': 'e115', 'key': 0}
{'id': 'e114', 'key': 0}
{'id': 'e111', 'key': 0}
{'id': 'e112', 'key': 0}
{'id': 'e74', 'key':

In [67]:
# Base for measure is Mb/s
mmnt = {'b/s':1e-6, 'kb/s': 1e-3, 'Mb/s': 1, 'Gb/s':1e+3, 'Tb/s':1e+6}

In [68]:
def unit_measure(strr):
    strr = strr.replace(" ", "")
    i = strr.rfind('b/s') + 3
    strr = strr[:i]
    for m in mmnt.keys():
        if m in strr:
            pattern = '{:d}' + m
            k = parse(pattern, strr)
            if k != None:
                return k[0]*mmnt[m]
    return None

In [69]:
bflows = {}
for e in H.edges():
    data = H.get_edge_data(e[1], e[0])
    if data['LinkLabel'] != 'Ethernet':
        speed = data['LinkLabel']
    else:
        speed = "100Mb/s"
        data['LinkLabel'] = speed
    bandwidth = unit_measure(speed)
    step = bandwidth / 100.0
    back_flow = random.randrange(0.0, bandwidth, step)
    bflows[e] = {'BackgroundFlow': back_flow}
        
nx.set_edge_attributes(H, bflows)
# print("New data in network")
# print(H.edges(data = True))

In [70]:
bflows

{('0', '85'): {'BackgroundFlow': 70},
 ('1', '85'): {'BackgroundFlow': 134},
 ('1', '103'): {'BackgroundFlow': 80},
 ('2', '3'): {'BackgroundFlow': 30},
 ('2', '132'): {'BackgroundFlow': 42},
 ('3', '4'): {'BackgroundFlow': 90},
 ('3', '5'): {'BackgroundFlow': 126},
 ('4', '48'): {'BackgroundFlow': 57},
 ('5', '6'): {'BackgroundFlow': 128},
 ('6', '157'): {'BackgroundFlow': 156},
 ('7', '8'): {'BackgroundFlow': 276},
 ('7', '9'): {'BackgroundFlow': 86},
 ('7', '109'): {'BackgroundFlow': 120},
 ('7', '85'): {'BackgroundFlow': 68},
 ('7', '119'): {'BackgroundFlow': 117},
 ('8', '9'): {'BackgroundFlow': 66},
 ('9', '21'): {'BackgroundFlow': 231},
 ('9', '86'): {'BackgroundFlow': 95},
 ('10', '18'): {'BackgroundFlow': 24},
 ('10', '11'): {'BackgroundFlow': 45},
 ('10', '13'): {'BackgroundFlow': 64},
 ('11', '16'): {'BackgroundFlow': 33},
 ('11', '18'): {'BackgroundFlow': 122},
 ('12', '13'): {'BackgroundFlow': 159},
 ('12', '30'): {'BackgroundFlow': 45},
 ('14', '17'): {'BackgroundFlow': 1

In [71]:
def print_Graph(H):
    nodes = []
    for node in H.nodes():
        nodes.append({'id': node, 'label': node})
    edges = []
    for edge in H.edges():
        data = H.get_edge_data(edge[0], edge[1])
        perc = data['BackgroundFlow'] / unit_measure(data['LinkLabel'])
        color = 'GREEN'
        if perc <= 0.5:
            color = 'GREEN'
        elif perc <= 0.8:
            color = 'YELLOW'
        elif perc <= 1:
            color = 'RED'
        edges.append({'from': edge[1], 'to': edge[0], 
                      'dashes':False, 'label': (str)(unit_measure(data['LinkLabel'])) + 'Mb/s', 
                      'font':{'align': 'top'},
                      'color': color,
                      'width': 4})
#         else:
#             edges.append({'from': edge[1], 'to': edge[0],
#                           'dashes':True, 'label': '0Mb/s', 
#                           'font':{'align': 'top'},
#                           'color': 'BLACK'})
    return (nodes, edges)

In [72]:
nodes, edges = print_Graph(H)

In [73]:
# Transform the graph into a JSON graph
data = {"nodes":nodes, "edges":edges}
jsonGraph = json.dumps(data, indent=4)

# Send to Javascript
Javascript("""window.jsonGraph={};""".format(jsonGraph))

<IPython.core.display.Javascript object>

In [74]:
%%javascript
requirejs.config({
    paths: {
        vis: 'vis'
    }
});

require(['vis'], function(vis){
    
    // create a network
    var container = document.getElementById('mynetwork');
    
    var options = {
        width: '900px',
        height: '500px',
        edges:{
          shadow: true,
          smooth: true,
          background:{
            enabled: true,
            color: '#ff0000'
          }
        }
    }
    
    // We load the JSON graph we generated from iPython input
    var graph = window.jsonGraph;
    
    // Display Graph
    var network = new vis.Network(container, graph, options);
});

<IPython.core.display.Javascript object>

In [75]:
%%html
<div id="mynetwork"></div>

In [76]:
Bandwidth = pd.DataFrame(0, index=np.arange(len(H.nodes())), 
                            columns=np.arange(len(H.nodes())))
BackFlows = pd.DataFrame(0, index=np.arange(len(H.nodes())), 
                            columns=np.arange(len(H.nodes())))

In [77]:
for edge in H.edges(data = True):
    data_edge = H.get_edge_data(edge[0], edge[1])
    bndw = unit_measure(data_edge['LinkLabel'])
    bflw = data_edge['BackgroundFlow']
#     print(Bandwidth.shape)
#     print(edge)
#     print(Bandwidth[0][237])
    Bandwidth[int(edge[1])][int(edge[0])] = bndw
    Bandwidth[int(edge[0])][int(edge[1])] = bndw
    BackFlows[int(edge[1])][int(edge[0])] = bflw
    BackFlows[int(edge[0])][int(edge[1])] = bflw

In [78]:
Bandwidth.head(5)

,0,1,2,3,4,5,6,7,8,9,...,148,149,150,151,152,153,154,155,156,157
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,300,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,300,0,100,300,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,100,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [79]:
BackFlows.head(5)

,0,1,2,3,4,5,6,7,8,9,...,148,149,150,151,152,153,154,155,156,157
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,30,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,30,0,90,126,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,90,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [80]:
from itertools import product

def print_answer(correct, answ, V, K, M):
    if correct == "correct":
        for i, j, k, t in product(V, V, K, M):
            if (answ[i][j][k][t] != None) and (answ[i][j][k][t] > 0.0):
                print("answer[{}][{}][{}][{}] = {}".format(i, j, k, t, answ[i][j][k][t]))
    else:
        print(correct)

In [96]:
Contracts = [{'volume': 800, 't_s': 5, 't':  80, 'A':87, 'B': 105},
             {'volume': 900, 't_s': 33, 't':  100, 'A': 65, 'B': 30},
             {'volume': 900, 't_s': 140, 't': 100, 'A': 42, 'B': 1}]

In [97]:
def set_time(t0, T, Contracts):
    t = {}
    points = set()
    points.add(t0)
    points.add(T)
    for c in Contracts:
        points.add(c['t_s'])
        points.add(c['t_s'] + c['t'])
    points = sorted(list(points))
    for i in range(len(points) - 1):
        e = points[i + 1] - points[i]
        C = set()
        for c in range(len(Contracts)):
            if points[i] >= Contracts[c]['t_s'] and points[i] < Contracts[c]['t_s'] + Contracts[c]['t']:
                C.add(c)
        t[i] = {'em': e, 'Ctm': C}
    return points, t

In [98]:
mmin = 10000000
mmax = 0
for c in Contracts:
    if c['t_s'] < mmin:
        mmin = c['t_s']
    if c['t_s'] + c['t'] > mmax:
        mmax = c['t_s'] + c['t']

In [99]:
V = set(range(Bandwidth.shape[0])) # число вершин
K = set(range(len(Contracts)))     # количество контрактов К
# t0 = 0                             # начало отсчета t0
# T = 1000                           # конец отсчета в сек
t0 = mmin
T = mmax
timeline, Intervals = set_time(t0, T, Contracts)
M = set(range(len(Intervals)))
timeline, Intervals

([5, 33, 85, 133, 140, 240],
 {0: {'em': 28, 'Ctm': {0}},
  1: {'em': 52, 'Ctm': {0, 1}},
  2: {'em': 48, 'Ctm': {1}},
  3: {'em': 7, 'Ctm': set()},
  4: {'em': 100, 'Ctm': {2}}})

In [100]:
import ipynb.fs.full.SolvePyomo as spyomo
import ipynb.fs.full.SolveMIP as mip

In [101]:
correct_pyomo, answ_pyomo, time_pyomo = spyomo.evaluate_model(V, K, M, Contracts, Intervals, Bandwidth, BackFlows)
print("Total time: ", time_pyomo)
print_answer(correct_pyomo, answ_pyomo, V, K, M)

Total time:  93.65331363677979
answer[2][132][0][0] = 10.0
answer[2][132][0][1] = 10.0
answer[2][132][2][4] = 9.0
answer[3][2][0][0] = 10.0
answer[3][2][0][1] = 10.0
answer[3][2][2][4] = 9.0
answer[5][3][0][0] = 10.0
answer[5][3][0][1] = 10.0
answer[5][3][2][4] = 9.0
answer[6][5][0][0] = 10.0
answer[6][5][0][1] = 10.0
answer[6][5][2][4] = 9.0
answer[42][87][2][4] = 9.0
answer[65][126][1][1] = 9.0
answer[65][126][1][2] = 9.0
answer[78][127][1][1] = 9.0
answer[78][127][1][2] = 9.0
answer[87][143][0][0] = 10.0
answer[87][143][0][1] = 10.0
answer[87][143][2][4] = 9.0
answer[96][120][1][1] = 9.0
answer[96][120][1][2] = 9.0
answer[102][105][0][0] = 10.0
answer[102][105][0][1] = 10.0
answer[103][1][2][4] = 9.0
answer[103][102][0][0] = 10.0
answer[103][102][0][1] = 10.0
answer[120][78][1][1] = 9.0
answer[120][78][1][2] = 9.0
answer[122][124][1][1] = 9.0
answer[122][124][1][2] = 9.0
answer[124][131][1][1] = 9.0
answer[124][131][1][2] = 9.0
answer[126][96][1][1] = 9.0
answer[126][96][1][2] = 9.0

In [102]:
def send_toJS(t, K, H, answ):
    nodes = []
    for node in H.nodes():
        nodes.append({'id': node, 'label': node})
    edges = []
    for edge in H.edges():
        data = H.get_edge_data(edge[1], edge[0])
        if data['LinkLabel'] != 'Ethernet':
            k_flow = 0
            label = ''
            for k in K:
                if answ[int(edge[0])][int(edge[1])][k][t] != 0:
                    k_flow += answ[int(edge[0])][int(edge[1])][k][t]
                    label += ', №' + (str)(k)
                if answ[int(edge[1])][int(edge[0])][k][t] != 0:
                    k_flow += answ[int(edge[1])][int(edge[0])][k][t]
                    label += ', №' + (str)(k)
            color = 'BLUE'
            if k_flow != 0:
                perc = (k_flow + BackFlows[int(edge[0])][int(edge[1])]) / Bandwidth[int(edge[0])][int(edge[1])]
                label += "\n" + (str)(k_flow) + "Mb/s"
                if perc <= 0.5:
                    color = 'GREEN'
                elif perc <= 0.8:
                    color = 'YELLOW'
                elif perc <= 1:
                    color = 'RED'        
            edges.append({'from': edge[1], 'to': edge[0], 
                          'dashes':False, 'label': label[2:],
                          'font':{'align': 'top'},
                          'color': color,
                          'width': 4})
        else:
            edges.append({'from': edge[1], 'to': edge[0],
                          'dashes':True, 'label': '0Mb/s', 
                          'font':{'align': 'top'},
                          'color': 'BLACK'})
            
    # Transform the graph into a JSON graph
    data = {"nodes":nodes, "edges":edges}
    jsonGraph = json.dumps(data, indent=4)

    return jsonGraph

In [103]:
t = 1
jsonGraph = send_toJS(t, K, H, answ_pyomo)
Javascript("""window.jsonGraph={};""".format(jsonGraph))

<IPython.core.display.Javascript object>

In [104]:
%%javascript
requirejs.config({
    paths: {
        vis: 'vis'
    }
});

require(['vis'], function(vis){
    
    // create a network
    var container = document.getElementById('resultnetwork_pyomo');
    
    var options = {
        width: '1000px',
        height: '400px',
        edges:{
          shadow: true,
          smooth: true,
          background:{
            enabled: true,
            color: '#ff0000'
          }
        }
    }
    
    // We load the JSON graph we generated from iPython input
    var graph = window.jsonGraph;
    
    // Display Graph
    var network = new vis.Network(container, graph, options);
});

<IPython.core.display.Javascript object>

In [105]:
%%html
<div id="resultnetwork_pyomo"></div>

In [106]:
correct_mip, answ_mip, time_mip = mip.evaluate_model(V, K, M, Contracts, Intervals, Bandwidth, BackFlows)
print("Total time: ", time_mip)
print_answer(correct_mip, answ_mip, V, K, M)

Total time:  50.90224647521973
answer[2][132][0][0] = 10.0
answer[2][132][0][1] = 10.0
answer[2][132][2][4] = 9.0
answer[3][2][0][0] = 10.0
answer[3][2][0][1] = 10.0
answer[3][2][2][4] = 9.0
answer[5][3][0][0] = 10.0
answer[5][3][0][1] = 10.0
answer[5][3][2][4] = 9.0
answer[6][5][0][0] = 10.0
answer[6][5][0][1] = 10.0
answer[6][5][2][4] = 9.0
answer[42][87][2][4] = 9.0
answer[65][126][1][1] = 9.0
answer[65][126][1][2] = 9.0
answer[78][127][1][1] = 9.0
answer[78][127][1][2] = 9.0
answer[87][143][0][0] = 10.0
answer[87][143][0][1] = 10.0
answer[87][143][2][4] = 9.0
answer[96][120][1][1] = 9.0
answer[96][120][1][2] = 9.0
answer[102][105][0][0] = 10.0
answer[102][105][0][1] = 10.0
answer[103][1][2][4] = 9.0
answer[103][102][0][0] = 10.0
answer[103][102][0][1] = 10.0
answer[120][78][1][1] = 9.0
answer[120][78][1][2] = 9.0
answer[122][124][1][1] = 9.0
answer[122][124][1][2] = 9.0
answer[124][131][1][1] = 9.0
answer[124][131][1][2] = 9.0
answer[126][96][1][1] = 9.0
answer[126][96][1][2] = 9.0

In [107]:
(answ_mip == answ_pyomo).all()

True

In [108]:
t = 0
jsonGraph = send_toJS(t, K, H, answ_mip)
Javascript("""window.jsonGraph={};""".format(jsonGraph))

<IPython.core.display.Javascript object>

In [109]:
%%javascript
requirejs.config({
    paths: {
        vis: 'vis'
    }
});

require(['vis'], function(vis){
    
    // create a network
    var container = document.getElementById('resultnetwork_mip');
    
    var options = {
        width: '800px',
        height: '400px',
        edges:{
          shadow: true,
          smooth: true,
          background:{
            enabled: true,
            color: '#ff0000'
          }
        }
    }
    
    // We load the JSON graph we generated from iPython input
    var graph = window.jsonGraph;
    
    // Display Graph
    var network = new vis.Network(container, graph, options);
});

<IPython.core.display.Javascript object>

In [110]:
%%html
<div id="resultnetwork_mip"></div>